In [1]:
import sys
sys.path.append("../../.")

import mlflow
from mlflow import MlflowClient
from src.utils.logger import get_logger
from services.embedding_service.app.utils.mlflow_utils import log_deployment_ready_model

logger = get_logger(__name__)

MLFLOW_TRACKING_URI = "http://localhost:5000/"
MODEL_NAME = "all-MiniLM-L12-v2"
EXPERIMENT_NAME = "deployment-ready-embedding-model"
ALIAS = "champion"

mlflow.set_tracking_uri(uri=MLFLOW_TRACKING_URI)
client = MlflowClient()

/home/joshuale/miniconda3/envs/local-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the model version using a model URI
model_uri = f"models:/{MODEL_NAME}@{ALIAS}"

try:
    logger.debug("Loading registered model from MLflow registry...")
    model = mlflow.sentence_transformers.load_model(model_uri)
    logger.debug("Embedding model loaded successfully.")
except Exception as e:
    logger.warning(
        f"Embeding model URI {model_uri} not found. Download, log and register..."
    )
    model_info = log_deployment_ready_model(
        experiment_name=EXPERIMENT_NAME, model_name=MODEL_NAME, alias=ALIAS
    )

    mlflow.register_model(model_uri=model_info.model_uri, name=MODEL_NAME)
    latest_version = client.get_registered_model(name=MODEL_NAME).latest_versions[0]
    client.set_registered_model_alias(
        name=MODEL_NAME, alias=ALIAS, version=latest_version.version
    )

2025-11-04 02:32:53 DEBUG    Loading registered model from MLflow registry...

2025/11/04 02:34:54 INFO mlflow.sentence_transformers: 'models:/all-MiniLM-L12-v2@champion' resolved as 'mlflow-artifacts:/4/models/m-6398fa4bdde54276a6fa0cc1a98244d9/artifacts'


2025-11-04 02:34:54 DEBUG    Embedding model loaded successfully.